In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/glove6b100dtxt/glove.6B.100d.txt
/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
!pip install torch==2.2.1 torchtext==0.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.1 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import re
from tqdm import tqdm
tqdm.pandas()

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
df = pd.read_csv('//kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
# Initialize stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Apply preprocessing
df['clean_review'] = df['review'].progress_apply(preprocess_text)
df.head()

100%|██████████| 50000/50000 [00:49<00:00, 1011.80it/s]


,review,sentiment,clean_review
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, oz, epis..."
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, filming, techn..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,negative,"[basically, theres, family, little, boy, jake,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, matteis, love, time, money, visually,..."


In [8]:
label_dict = {'positive': 1, 'negative': 0}
df['label'] = df['sentiment'].map(label_dict)
df.head()

,review,sentiment,clean_review,label
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, oz, epis...",1
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, filming, techn...",1
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su...",1
3,Basically there's a family where a little boy ...,negative,"[basically, theres, family, little, boy, jake,...",0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, matteis, love, time, money, visually,...",1


In [9]:
X = df['clean_review']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from collections import Counter

# Build vocabulary
all_words = [word for tokens in X_train for word in tokens]
word_counts = Counter(all_words)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
vocab_size = len(vocab)

print(f"Vocab size: {vocab_size}")

# Word to index mapping
word_to_idx = {word: idx+2 for idx, word in enumerate(vocab)}
word_to_idx["<PAD>"] = 0
word_to_idx["<UNK>"] = 1

# Index to word mapping
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

Vocab size: 186211


In [12]:
def encode_text(text):
    return [word_to_idx.get(word, word_to_idx["<UNK>"]) for word in text]

# Encode training and testing data
X_train_enc = X_train.apply(encode_text)
X_test_enc = X_test.apply(encode_text)

In [14]:
def pad_sequences(sequences, max_len):
    sequences = [torch.tensor(seq[:max_len]) for seq in sequences]
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=word_to_idx["<PAD>"])
    return sequences_padded

max_len = 200  # Define max length

X_train_padded = pad_sequences(X_train_enc, max_len)
X_test_padded = pad_sequences(X_test_enc, max_len)

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

In [15]:
train_dataset = torch.utils.data.TensorDataset(X_train_padded, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_padded, y_test_tensor)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
embedding_dim = 100
glove_path = '/kaggle/input/glove6b100dtxt/glove.6B.100d.txt'

# Load GloVe embeddings
embeddings_index = {}
with open(glove_path, encoding='utf8') as f:
    for line in tqdm(f, desc="Loading GloVe"):
        values = line.strip().split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coeffs

print(f"Loaded {len(embeddings_index)} word vectors.")

Loading GloVe: 400000it [00:12, 32936.86it/s]

Loaded 400000 word vectors.


In [17]:
embedding_matrix = np.zeros((vocab_size + 2, embedding_dim))

In [18]:
for word, idx in word_to_idx.items():
    if word in embeddings_index:
        embedding_matrix[idx] = embeddings_index[word]
    else:
        # Initialize random embedding for words not in GloVe
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [19]:
# Convert to tensor
embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float32)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: torch.Size([186213, 100])


In [20]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, n_layers, drop_prob=0.5):
        super(RNNClassifier, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape

        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        embeds = self.embedding(x)
        rnn_out, hidden = self.rnn(embeds)
        out = self.dropout(hidden[-1])
        out = self.fc(out)
        return out

In [21]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, n_layers, drop_prob=0.5):
        super(LSTMClassifier, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape

        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embeds)
        out = self.dropout(hidden[-1])
        out = self.fc(out)
        return out

In [22]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs, device):
    model.to(device)
    best_valid_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_acc = 0

        for texts, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            texts = texts.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            predictions = model(texts)
            loss = criterion(predictions.squeeze(), labels.float())
            preds = torch.round(torch.sigmoid(predictions.squeeze()))
            acc = (preds == labels).float().mean()

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        valid_loss, valid_acc = evaluate_model(model, valid_loader, criterion, device)

        print(f'\nEpoch {epoch+1}/{epochs}')
        print(f'Train Loss: {epoch_loss/len(train_loader):.3f} | Train Acc: {epoch_acc/len(train_loader):.3f}')
        print(f'Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.3f}')

    return model

In [23]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for texts, labels in data_loader:
            texts = texts.to(device)
            labels = labels.to(device)

            predictions = model(texts)
            loss = criterion(predictions.squeeze(), labels.float())
            preds = torch.round(torch.sigmoid(predictions.squeeze()))
            acc = (preds == labels).float().mean()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(data_loader), epoch_acc / len(data_loader)

In [25]:
# Set parameters
hidden_dim = 128
output_dim = 1
n_layers = 2
epochs = 5
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model, criterion and optimizer
rnn_model = RNNClassifier(embedding_matrix, hidden_dim, output_dim, n_layers)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)

# Train the model
trained_rnn_model = train_model(rnn_model, train_loader, test_loader, criterion, optimizer,epochs, device)

Epoch 1/5: 100%|██████████| 625/625 [12:58<00:00,  1.24s/it]



Epoch 1/5
Train Loss: 0.699 | Train Acc: 0.502
Val. Loss: 0.692 |  Val. Acc: 0.508


Epoch 2/5: 100%|██████████| 625/625 [12:50<00:00,  1.23s/it]



Epoch 2/5
Train Loss: 0.693 | Train Acc: 0.511
Val. Loss: 0.690 |  Val. Acc: 0.514


Epoch 3/5: 100%|██████████| 625/625 [10:44<00:00,  1.03s/it]



Epoch 3/5
Train Loss: 0.694 | Train Acc: 0.512
Val. Loss: 0.692 |  Val. Acc: 0.530


Epoch 4/5: 100%|██████████| 625/625 [12:39<00:00,  1.22s/it]



Epoch 4/5
Train Loss: 0.693 | Train Acc: 0.520
Val. Loss: 0.691 |  Val. Acc: 0.531


Epoch 5/5: 100%|██████████| 625/625 [12:07<00:00,  1.16s/it]



Epoch 5/5
Train Loss: 0.693 | Train Acc: 0.524
Val. Loss: 0.692 |  Val. Acc: 0.531


In [26]:
# Initialize model, criterion and optimizer
lstm_model = LSTMClassifier(embedding_matrix, hidden_dim, output_dim, n_layers)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)

# Train the model
trained_lstm_model = train_model(lstm_model, train_loader, test_loader, criterion, optimizer, epochs, device)

Epoch 1/5: 100%|██████████| 625/625 [21:47<00:00,  2.09s/it]



Epoch 1/5
Train Loss: 0.690 | Train Acc: 0.513
Val. Loss: 0.693 |  Val. Acc: 0.508


Epoch 2/5: 100%|██████████| 625/625 [12:10<00:00,  1.17s/it]



Epoch 2/5
Train Loss: 0.686 | Train Acc: 0.523
Val. Loss: 0.678 |  Val. Acc: 0.532


Epoch 3/5: 100%|██████████| 625/625 [07:00<00:00,  1.49it/s]



Epoch 3/5
Train Loss: 0.586 | Train Acc: 0.693
Val. Loss: 0.470 |  Val. Acc: 0.812


Epoch 4/5: 100%|██████████| 625/625 [06:11<00:00,  1.68it/s]



Epoch 4/5
Train Loss: 0.394 | Train Acc: 0.845
Val. Loss: 0.359 |  Val. Acc: 0.838


Epoch 5/5: 100%|██████████| 625/625 [06:03<00:00,  1.72it/s]



Epoch 5/5
Train Loss: 0.273 | Train Acc: 0.897
Val. Loss: 0.333 |  Val. Acc: 0.872


In [30]:
def test_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for texts, labels in data_loader:
            texts = texts.to(device)
            labels = labels.to(device)

            predictions = model(texts)
            preds = torch.round(torch.sigmoid(predictions.squeeze()))

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=['Negative', 'Positive'])

    print(f'Accuracy: {acc:.3f}')
    print(report)

In [31]:
print("RNN Model Evaluation:")
test_model(trained_rnn_model, test_loader, device)

print("LSTM Model Evaluation:")
test_model(trained_lstm_model, test_loader, device)

RNN Model Evaluation:
Accuracy: 0.562
              precision    recall  f1-score   support

    Negative       0.55      0.62      0.58      4961
    Positive       0.57      0.51      0.54      5039

    accuracy                           0.56     10000
   macro avg       0.56      0.56      0.56     10000
weighted avg       0.56      0.56      0.56     10000

LSTM Model Evaluation:
Accuracy: 0.872
              precision    recall  f1-score   support

    Negative       0.88      0.85      0.87      4961
    Positive       0.86      0.89      0.88      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [32]:
class RNNClassifierOnTheFly(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, drop_prob=0.5):
        super(RNNClassifierOnTheFly, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        embeds = self.embedding(x)
        rnn_out, hidden = self.rnn(embeds)
        out = self.dropout(hidden[-1])
        out = self.fc(out)
        return out

In [33]:
class LSTMClassifierOnTheFly(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, drop_prob=0.5):
        super(LSTMClassifierOnTheFly, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embeds)
        out = self.dropout(hidden[-1])
        out = self.fc(out)
        return out

In [34]:
# Set parameters
embedding_dim = 100  # Same as GloVe
hidden_dim = 128
output_dim = 1
n_layers = 2
epochs = 5
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model, criterion, and optimizer
rnn_model_otf = RNNClassifierOnTheFly(len(word_to_idx), embedding_dim, hidden_dim, output_dim, n_layers)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(rnn_model_otf.parameters(), lr=learning_rate)

# Train the model
trained_rnn_model_otf = train_model(rnn_model_otf, train_loader, test_loader, criterion, optimizer, epochs, device)

Epoch 1/5: 100%|██████████| 625/625 [16:01<00:00,  1.54s/it]



Epoch 1/5
Train Loss: 0.701 | Train Acc: 0.500
Val. Loss: 0.695 |  Val. Acc: 0.505


Epoch 2/5: 100%|██████████| 625/625 [17:34<00:00,  1.69s/it]



Epoch 2/5
Train Loss: 0.694 | Train Acc: 0.503
Val. Loss: 0.695 |  Val. Acc: 0.501


Epoch 3/5: 100%|██████████| 625/625 [14:43<00:00,  1.41s/it]



Epoch 3/5
Train Loss: 0.692 | Train Acc: 0.517
Val. Loss: 0.695 |  Val. Acc: 0.500


Epoch 4/5: 100%|██████████| 625/625 [13:06<00:00,  1.26s/it]



Epoch 4/5
Train Loss: 0.681 | Train Acc: 0.530
Val. Loss: 0.707 |  Val. Acc: 0.501


Epoch 5/5: 100%|██████████| 625/625 [12:51<00:00,  1.23s/it]



Epoch 5/5
Train Loss: 0.664 | Train Acc: 0.545
Val. Loss: 0.726 |  Val. Acc: 0.502


In [ ]:
# Initialize model, criterion, and optimizer
lstm_model_otf = LSTMClassifierOnTheFly(len(word_to_idx), embedding_dim, hidden_dim, output_dim, n_layers)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(lstm_model_otf.parameters(), lr=learning_rate)

# Train the model
trained_lstm_model_otf = train_model(lstm_model_otf, train_loader, test_loader, criterion, optimizer, epochs, device)

Epoch 1/5:  84%|████████▍ | 526/625 [26:58<02:55,  1.78s/it]

In [ ]:
print("RNN Model with On-the-Fly Embeddings Evaluation:")
test_model(trained_rnn_model_otf, test_loader, device)

print("LSTM Model with On-the-Fly Embeddings Evaluation:")
test_model(trained_lstm_model_otf, test_loader, device)